Installing dependencies

In [ ]:
!pip install langchain-huggingface
!pip install langchain-community langchain
!pip install python-dotenv
!pip install jq
!pip install -qU langchain-pinecone pinecone-notebooks
!pip install gradio
!pip install pypdf2

In [ ]:
json_file = 

Importing necessary libraries

In [3]:
# To connect drive
from google.colab import drive
# To access file from .env
from dotenv import load_dotenv
import os
# To login Huggingface
from huggingface_hub import login
# JSON data loader
from langchain_community.document_loaders import JSONLoader
# Importing Embedding
from langchain_huggingface import HuggingFaceEmbeddings
# To access LLM model
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
# connect with Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
# Formatting data to store in PC
import json
from langchain.schema import Document
from uuid import uuid4
# prompt template
from langchain.prompts import PromptTemplate
# To format output
from IPython.display import display, Markdown

# for gradio
import gradio as gr
from PIL import Image
import PyPDF2

Loading keys from .env

In [ ]:
data_file_path = 'restaurant_data.json'
load_dotenv()
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
from huggingface_hub import login
login(HUGGINGFACE_API_KEY)

Loading JSON file

In [7]:
loader = JSONLoader(
    file_path = data_file_path ,
    jq_schema = '.',
    text_content = False)

data = loader.load()

Choosing LLM model

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

Embedding and storing it Vector Database

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "restaurant-vector-index"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
index = pc.Index(index_name)

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
documents = []
for entry in data:
    # Parse the JSON content from the page_content
    restaurant_info = json.loads(entry.page_content)

    # Create a Document with structured information
    documents.append(Document(
        page_content=json.dumps(restaurant_info),  # You can format this as needed
        metadata={
            'source': entry.metadata['source'],
            'seq_num': entry.metadata['seq_num']
        }
    ))

# Generate unique IDs for each document
uuids = [str(uuid4()) for _ in range(len(documents))]

# Add documents to Pinecone
vector_store.add_documents(documents=documents, ids=uuids)

User Define query

In [10]:
user_query = 'is you open on weekends? if open at what time? is there parking available'

Getting result from Vector storage

In [11]:
results = vector_store.similarity_search(
    query=user_query,
    k=1
)

Getting response from LLM model

In [12]:
prompt_template = PromptTemplate(
    input_variables=["results", "query"],
    # template='''Given the data: {results}, please respond to the query: "{query}" with the relevant information only.'''
    template='''Given the data: {results}, please respond to the query: "{query}".'''
)

# Create the formatted prompt
formatted_prompt = prompt_template.format(results=results, query=user_query)

# Create the messages
messages = [
    {"role": "human", "content": formatted_prompt}
]

# Invoke the chat with the messages
response = chat.invoke(messages)


Displaying Output

In [ ]:
display(Markdown(response.content))